In [ ]:
"""Functions for plugging into Pipulate-frameworks for conducting SEO investigations."""

In [ ]:
import requests, re
from collections import namedtuple
Response = namedtuple('Response', 'ok status_code text')

In [ ]:
def url(passed_in_func):
    """Decorator for functions like Title to pre-extract html text."""
    def requests_wrapper(**row_dict):
        html = row_dict['response'].text
        return passed_in_func(html=html)
    return requests_wrapper

In [ ]:
def text_node(html, tag):
    pattern = r'<{0}\s?>(.*?)</{0}\s?>'.format(tag.lower())
    compiled = re.compile(pattern=pattern, flags=re.DOTALL)
    matches = compiled.findall(string=html)
    if matches:
        text = matches[0].strip()
        return Response(ok=True, status_code='200', text=text)
    else:
        return Response(ok=True, status_code='200', text=None)

In [ ]:
@url
def Title(html):
    return text_node(html=html, tag="title")

In [ ]:
if __name__ == '__main__':
    test_url = 'http://mikelev.in/'
    response = requests.get(test_url)
    print(Title(url=test_url, response=response))